# This is a prototype of using Best Track data as input to the storm surge workflow

In [1]:
#%matplotlib notebook

In [2]:
%matplotlib inline

In [3]:
import numpy as np
# -------------------------------------------------------------------------
# Const
# -------------------------------------------------------------------------
nm2m=1852. # 1 nautical mile to meters
kt2ms=nm2m/3600.  # knots to m/s
omega=2*np.pi/(3600.*24.) # angular speed omega=2pi*f(=frequency of earth : 1 cycle per day) 2pi* 1 / day in seconds
rhoa=1.15 #air density  Kg/m^3
radius=6378388 #137. # earth's radius according to WGS 84
deg2m=np.pi*radius/180.  # ds on cicle equals ds=r*dth - dth=pi/180
pn=101000.  # Atmospheric pressure [N/m^2] (101KPa - enviromental pressure)

tetaNE=45. #mean angle [degrees] of North Eastern quadrant
tetaNW=135. #        "              North Western
tetaSW=225. #        "              South West
tetaSE=315. #        "              South East

maxR=500.e3  # maximum radius of TC [m] (500Km)

In [4]:
from netCDF4 import Dataset

In [5]:
import glob

In [6]:
import re

In [7]:
import datetime

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import mplleaflet

In [14]:
path='../BestTrack/'

In [15]:
data=pd.read_csv(path+'Allstorms.ibtracs_all.v03r09.csv', header=1, low_memory=False)

In [16]:
data['Basin'] = data.loc[:,'Basin'].str.strip() # strip basin reference

In [17]:
data.head()

,Serial_Num,Season,Num,Basin,Sub_basin,Name,ISO_time,Nature,Latitude,Longitude,...,jtwc_??_wrad34_rad3,jtwc_??_wrad34_rad4,jtwc_??_wrad50_rad1,jtwc_??_wrad50_rad2,jtwc_??_wrad50_rad3,jtwc_??_wrad50_rad4,jtwc_??_wrad64_rad1,jtwc_??_wrad64_rad2,jtwc_??_wrad64_rad3,jtwc_??_wrad64_rad4
0,NaN,Year,#,BB,BB,NaN,YYYY-MM-DD HH:MM:SS,NaN,deg_north,deg_east,...,nmile,nmile,nmile,nmile,nmile,nmile,nmile,nmile,nmile,nmile
1,1842298N11080,1842,01,NI,BB,NOT NAMED,1842-10-25 06:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
2,1842298N11080,1842,01,NI,BB,NOT NAMED,1842-10-25 12:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
3,1842298N11080,1842,01,NI,AS,NOT NAMED,1842-10-25 18:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
4,1842298N11080,1842,01,NI,AS,NOT NAMED,1842-10-26 00:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000


# ANALYSIS STARTS HERE

In [18]:
hurName='INGRID'
year = '2013'

In [19]:
hurdata = data[data['Name'].str.contains(hurName,na=False) & data['ISO_time'].str.contains(year,na=False)]

In [20]:
hurdata.iloc[0]

Serial_Num                     2013255N19268
Season                                  2013
Num                                       10
Basin                                     NA
Sub_basin                                 GM
Name                                  INGRID
ISO_time                 2013-09-12 06:00:00
Nature                                    DS
Latitude                               19.30
Longitude                             -92.20
Wind(WMO)                               25.0
Pres(WMO)                             1006.0
Center                                  atcf
Wind(WMO) Percentile                   4.993
Pres(WMO) Percentile                  26.787
Track_type                              main
Latitude_for_mapping                   19.30
Longitude_for_mapping                 -92.20
Current Basin                            NaN
hurdat_atl_lat                          19.3
hurdat_atl_lon                         -92.2
hurdat_atl_grade                      -999.0
hurdat_atl

In [21]:
hurdata.shape

(22, 200)

In [22]:
ntc=hurdata['Basin'].unique()

In [23]:
ntc

array(['NA'], dtype=object)

In [24]:
hur1=hurdata.loc[hurdata['Basin']=='NA'].copy()

In [25]:
hur1

,Serial_Num,Season,Num,Basin,Sub_basin,Name,ISO_time,Nature,Latitude,Longitude,...,jtwc_??_wrad34_rad3,jtwc_??_wrad34_rad4,jtwc_??_wrad50_rad1,jtwc_??_wrad50_rad2,jtwc_??_wrad50_rad3,jtwc_??_wrad50_rad4,jtwc_??_wrad64_rad1,jtwc_??_wrad64_rad2,jtwc_??_wrad64_rad3,jtwc_??_wrad64_rad4
336297,2013255N19268,2013,10,NA,GM,INGRID,2013-09-12 06:00:00,DS,19.30,-92.20,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
336298,2013255N19268,2013,10,NA,GM,INGRID,2013-09-12 12:00:00,DS,19.60,-92.90,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
336299,2013255N19268,2013,10,NA,GM,INGRID,2013-09-12 18:00:00,TS,19.70,-93.60,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
336300,2013255N19268,2013,10,NA,GM,INGRID,2013-09-13 00:00:00,TS,19.70,-94.10,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
336301,2013255N19268,2013,10,NA,GM,INGRID,2013-09-13 06:00:00,TS,19.60,-94.60,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
336302,2013255N19268,2013,10,NA,GM,INGRID,2013-09-13 12:00:00,TS,19.30,-95.10,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
336303,2013255N19268,2013,10,NA,GM,INGRID,2013-09-13 18:00:00,TS,19.10,-95.30,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
336304,2013255N19268,2013,10,NA,GM,INGRID,2013-09-14 00:00:00,TS,19.30,-95.30,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
336305,2013255N19268,2013,10,NA,GM,INGRID,2013-09-14 06:00:00,TS,19.60,-95.10,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
336306,2013255N19268,2013,10,NA,GM,INGRID,2013-09-14 12:00:00,TS,20.30,-94.60,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000


In [26]:
hur1.dropna()

,Serial_Num,Season,Num,Basin,Sub_basin,Name,ISO_time,Nature,Latitude,Longitude,...,jtwc_??_wrad34_rad3,jtwc_??_wrad34_rad4,jtwc_??_wrad50_rad1,jtwc_??_wrad50_rad2,jtwc_??_wrad50_rad3,jtwc_??_wrad50_rad4,jtwc_??_wrad64_rad1,jtwc_??_wrad64_rad2,jtwc_??_wrad64_rad3,jtwc_??_wrad64_rad4
336314,2013255N19268,2013,10,NA,GM,INGRID,2013-09-16 11:15:00,TS,23.70,-97.70,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [27]:
hur1[['Longitude','Latitude']]=hur1.loc[:,['Longitude','Latitude']].apply(pd.to_numeric)

Check if we cross International Date Line (IDL)

In [28]:
sig=np.sign(hur1.Longitude.values)
sig1=sig[0]
m=sig != sig1

In [29]:
if sum(m)>0:
# adjust the lon values going from -180:180
        if sig1 > 0:
                hur1.loc[hur1.Longitude < 0, ['Longitude']] = hur1.Longitude+360.
        elif sig1 < 0:
                hur1.loc[hur1.Longitude > 0, ['Longitude']] = hur1.Longitude-360.



In [30]:
plt.plot(hur1['Longitude'],hur1['Latitude'],'o--')
mplleaflet.display()

In [31]:
idf=hur1.iloc[0].Serial_Num

In [35]:
filename=glob.glob('../BestTrack/ibtracs/{}*'.format(idf))

In [37]:
dat=Dataset(filename[0])

In [38]:
dat.variables.keys()

[u'nobs_wmo',
 u'nobs',
 u'genesis_basin',
 u'num_basins',
 u'season',
 u'source',
 u'track_from_source',
 u'name_from_source',
 u'time_wmo',
 u'isotime_wmo',
 u'source_wmo',
 u'averaging_time',
 u'lat_wmo',
 u'lon_wmo',
 u'wind_wmo',
 u'pres_wmo',
 u'basin_wmo',
 u'sub_basin_wmo',
 u'nature_wmo',
 u'dist2land_wmo',
 u'landfall_wmo',
 u'pres_perc_wmo',
 u'wind_perc_wmo',
 u'time',
 u'isotime',
 u'lat_for_mapping',
 u'lon_for_mapping',
 u'wind_for_mapping',
 u'pres_for_mapping',
 u'nature_for_mapping',
 u'dist2land',
 u'landfall',
 u'lat_from_source',
 u'lon_from_source',
 u'wind_from_source',
 u'wind_avg_period',
 u'wind_perc_from_source',
 u'pres_from_source',
 u'pres_perc_from_source',
 u'basin',
 u'sub_basin',
 u'pos_stdev',
 u'pos_num',
 u'pos_flag',
 u'wind_quality',
 u'wind_stdev',
 u'wind_condition',
 u'wind_num',
 u'wind_flag',
 u'wind_maximum',
 u'wind_minimum',
 u'wind_median',
 u'pres_quality',
 u'pres_stdev',
 u'pres_condition',
 u'pres_flag',
 u'pres_num',
 u'pres_maximum'

In [39]:
wradii=dat['atcf_wrad']

In [40]:
wradiinp=np.array(wradii)

In [41]:
w34= wradiinp[:,0,:]

In [42]:
w34ne=w34[:,0]
w34nw=w34[:,1]
w34se=w34[:,2]
w34sw=w34[:,3]

In [43]:
w50=wradiinp[:,1,:]

In [44]:
w50ne=w50[:,0]
w50nw=w50[:,1]
w50se=w50[:,2]
w50sw=w50[:,3]

In [45]:
w64=wradiinp[:,2,:]

In [46]:
w64ne=w64[:,0]
w64nw=w64[:,1]
w64se=w64[:,2]
w64sw=w64[:,3]

In [47]:
isot=dat['isotime']

In [48]:
isot1=[''.join(x) for x in isot]

In [49]:
time=[re.sub(r'[-: ]+','',x)[:-4] for x in isot1]

In [50]:
lat=dat['atcf_lat'][:]
lon=dat['atcf_lon'][:]
pcenter=dat['atcf_pres'][:]
vmaxh=dat['atcf_wind'][:] # 10 minute wind in Knots
rmaxh=dat['atcf_rmw'][:]*nm2m

In [51]:
dic={'t':time, 'lat':lat,'lon':lon,'pcenter':pcenter,'vmax':vmaxh, 'rmax':rmaxh, '34ne': w34ne, '34nw': w34nw, '34se':w34se, '34sw':w34sw, '50ne': w50ne, '50nw': w50nw, '50se':w50se, '50sw':w50sw,'64ne': w64ne, '64nw': w64nw, '64se':w64se, '64sw':w64sw} 

In [77]:
inpData=pd.DataFrame(dic)

In [78]:
inpData=inpData.apply(pd.to_numeric)

In [79]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax
0,0,0,0,0,0,0,0,0,0,0,0,0,19.299999,-92.199997,1006,222240.0,2013091206,25
1,0,0,0,0,0,0,0,0,0,0,0,0,19.600000,-92.900002,1005,222240.0,2013091212,30
2,0,0,0,0,0,0,0,0,0,0,0,0,19.700001,-93.599998,1004,166680.0,2013091218,30
3,0,0,0,0,0,0,0,0,0,0,0,0,19.700001,-94.099998,1003,166680.0,2013091300,30
4,0,0,0,0,0,0,0,0,0,0,0,0,19.600000,-94.599998,1002,166680.0,2013091306,30
5,40,0,40,40,0,0,0,0,0,0,0,0,19.299999,-95.099998,1000,74080.0,2013091312,35
6,50,30,40,40,0,0,0,0,0,0,0,0,19.100000,-95.300003,996,74080.0,2013091318,40
7,70,40,50,40,40,0,0,0,0,0,0,0,19.299999,-95.300003,993,55560.0,2013091400,50
8,70,60,50,40,40,0,0,0,0,0,0,0,19.600000,-95.099998,990,55560.0,2013091406,55
9,70,60,50,40,30,0,0,30,0,0,0,0,20.299999,-94.599998,989,37040.0,2013091412,60


In [80]:
inpData.loc[inpData['lat'] == 9.969210e+36, 'lat'] = np.nan
inpData.loc[inpData['lon'] == 9.969210e+36, 'lon'] = np.nan
inpData.loc[inpData['pcenter'] == -32767., 'pcenter'] = np.nan
inpData.loc[inpData['rmax'] == -60684484., 'rmax'] = np.nan
inpData.loc[inpData['vmax'] == -32767, 'vmax'] = np.nan
inpData.loc[inpData['34ne'] == -32767, '34ne']=np.nan
inpData.loc[inpData['34nw'] == -32767, '34nw']=np.nan
inpData.loc[inpData['34se'] == -32767, '34se']=np.nan
inpData.loc[inpData['34sw'] == -32767, '34sw']=np.nan
inpData.loc[inpData['50ne'] == -32767, '50ne']=np.nan
inpData.loc[inpData['50nw'] == -32767, '50nw']=np.nan
inpData.loc[inpData['50se'] == -32767, '50se']=np.nan
inpData.loc[inpData['50sw'] == -32767, '50sw']=np.nan
inpData.loc[inpData['64ne'] == -32767, '64ne']=np.nan
inpData.loc[inpData['64nw'] == -32767, '64nw']=np.nan
inpData.loc[inpData['64se'] == -32767, '64se']=np.nan
inpData.loc[inpData['64sw'] == -32767, '64sw']=np.nan

In [81]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.299999,-92.199997,1006.0,222240.0,2013091206,25.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.600000,-92.900002,1005.0,222240.0,2013091212,30.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.700001,-93.599998,1004.0,166680.0,2013091218,30.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.700001,-94.099998,1003.0,166680.0,2013091300,30.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.600000,-94.599998,1002.0,166680.0,2013091306,30.0
5,40.0,0.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.299999,-95.099998,1000.0,74080.0,2013091312,35.0
6,50.0,30.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.100000,-95.300003,996.0,74080.0,2013091318,40.0
7,70.0,40.0,50.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.299999,-95.300003,993.0,55560.0,2013091400,50.0
8,70.0,60.0,50.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.600000,-95.099998,990.0,55560.0,2013091406,55.0
9,70.0,60.0,50.0,40.0,30.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,20.299999,-94.599998,989.0,37040.0,2013091412,60.0


In [82]:
inpData.loc[inpData['34ne'] == -1, '34ne']=0
inpData.loc[inpData['34nw'] == -1, '34nw']=0
inpData.loc[inpData['34se'] == -1, '34se']=0
inpData.loc[inpData['34sw'] == -1, '34sw']=0
inpData.loc[inpData['50ne'] == -1, '50ne']=0
inpData.loc[inpData['50nw'] == -1, '50nw']=0
inpData.loc[inpData['50se'] == -1, '50se']=0
inpData.loc[inpData['50sw'] == -1, '50sw']=0
inpData.loc[inpData['64ne'] == -1, '64ne']=0
inpData.loc[inpData['64nw'] == -1, '64nw']=0
inpData.loc[inpData['64se'] == -1, '64se']=0
inpData.loc[inpData['64sw'] == -1, '64sw']=0

In [83]:
inpData=inpData.dropna()

In [84]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.299999,-92.199997,1006.0,222240.0,2013091206,25.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.600000,-92.900002,1005.0,222240.0,2013091212,30.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.700001,-93.599998,1004.0,166680.0,2013091218,30.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.700001,-94.099998,1003.0,166680.0,2013091300,30.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.600000,-94.599998,1002.0,166680.0,2013091306,30.0
5,40.0,0.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.299999,-95.099998,1000.0,74080.0,2013091312,35.0
6,50.0,30.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.100000,-95.300003,996.0,74080.0,2013091318,40.0
7,70.0,40.0,50.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.299999,-95.300003,993.0,55560.0,2013091400,50.0
8,70.0,60.0,50.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.600000,-95.099998,990.0,55560.0,2013091406,55.0
9,70.0,60.0,50.0,40.0,30.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,20.299999,-94.599998,989.0,37040.0,2013091412,60.0


In [85]:
dph=pn-inpData.pcenter*100. #compute Dp

In [86]:
dph

0      400.0
1      500.0
2      600.0
3      700.0
4      800.0
5     1000.0
6     1400.0
7     1700.0
8     2000.0
9     2100.0
10    2400.0
11    2700.0
12    2500.0
13    2100.0
14    2300.0
15    2100.0
16    2000.0
17    1900.0
18    1200.0
19     400.0
20     200.0
Name: pcenter, dtype: float64

In [87]:
inpData=inpData.assign(dp=dph) #add to DataFrame

In [88]:
inpData.iloc[3] #check one row

34ne       0.000000e+00
34nw       0.000000e+00
34se       0.000000e+00
34sw       0.000000e+00
50ne       0.000000e+00
50nw       0.000000e+00
50se       0.000000e+00
50sw       0.000000e+00
64ne       0.000000e+00
64nw       0.000000e+00
64se       0.000000e+00
64sw       0.000000e+00
lat        1.970000e+01
lon       -9.410000e+01
pcenter    1.003000e+03
rmax       1.666800e+05
t          2.013091e+09
vmax       3.000000e+01
dp         7.000000e+02
Name: 3, dtype: float64

### create inpData file

In [89]:
inpData['t']=pd.to_datetime(inpData['t'],format='%Y%m%d%H')

In [90]:
inpData.head()

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax,dp
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.299999,-92.199997,1006.0,222240.0,2013-09-12 06:00:00,25.0,400.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.600000,-92.900002,1005.0,222240.0,2013-09-12 12:00:00,30.0,500.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.700001,-93.599998,1004.0,166680.0,2013-09-12 18:00:00,30.0,600.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.700001,-94.099998,1003.0,166680.0,2013-09-13 00:00:00,30.0,700.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.600000,-94.599998,1002.0,166680.0,2013-09-13 06:00:00,30.0,800.0


In [91]:
inpData.t.iloc[0] #check format

Timestamp('2013-09-12 06:00:00')

In [92]:
inpData['time']=inpData['t']-inpData.t.iloc[0] #create Dt

In [93]:
inpData['time']=inpData['time'] / pd.Timedelta('1 hour') #convert to hours and add to DataFrame

In [94]:
inpData=inpData.set_index('time') 
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax,dp
time,,,,,,,,,,,,,,,,,,,
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.299999,-92.199997,1006.0,222240.0,2013-09-12 06:00:00,25.0,400.0
6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.600000,-92.900002,1005.0,222240.0,2013-09-12 12:00:00,30.0,500.0
12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.700001,-93.599998,1004.0,166680.0,2013-09-12 18:00:00,30.0,600.0
18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.700001,-94.099998,1003.0,166680.0,2013-09-13 00:00:00,30.0,700.0
24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.600000,-94.599998,1002.0,166680.0,2013-09-13 06:00:00,30.0,800.0
30.0,40.0,0.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.299999,-95.099998,1000.0,74080.0,2013-09-13 12:00:00,35.0,1000.0
36.0,50.0,30.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.100000,-95.300003,996.0,74080.0,2013-09-13 18:00:00,40.0,1400.0
42.0,70.0,40.0,50.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.299999,-95.300003,993.0,55560.0,2013-09-14 00:00:00,50.0,1700.0
48.0,70.0,60.0,50.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.600000,-95.099998,990.0,55560.0,2013-09-14 06:00:00,55.0,2000.0


## convert to SI

In [95]:
inpData['vmax']=inpData['vmax']*kt2ms

In [96]:
inpData[['64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']]=inpData[['64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']]*nm2m

In [97]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax,dp
time,,,,,,,,,,,,,,,,,,,
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.299999,-92.199997,1006.0,222240.0,2013-09-12 06:00:00,12.861111,400.0
6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.600000,-92.900002,1005.0,222240.0,2013-09-12 12:00:00,15.433333,500.0
12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.700001,-93.599998,1004.0,166680.0,2013-09-12 18:00:00,15.433333,600.0
18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.700001,-94.099998,1003.0,166680.0,2013-09-13 00:00:00,15.433333,700.0
24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.600000,-94.599998,1002.0,166680.0,2013-09-13 06:00:00,15.433333,800.0
30.0,74080.0,0.0,74080.0,74080.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.299999,-95.099998,1000.0,74080.0,2013-09-13 12:00:00,18.005556,1000.0
36.0,92600.0,55560.0,74080.0,74080.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.100000,-95.300003,996.0,74080.0,2013-09-13 18:00:00,20.577778,1400.0
42.0,129640.0,74080.0,92600.0,74080.0,74080.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.299999,-95.300003,993.0,55560.0,2013-09-14 00:00:00,25.722222,1700.0
48.0,129640.0,111120.0,92600.0,74080.0,74080.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.600000,-95.099998,990.0,55560.0,2013-09-14 06:00:00,28.294444,2000.0


## save inpData file

In [98]:
column_order=['lat','lon','dp','vmax','64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']

In [99]:
header=['lat','long','dp','vmax','64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']

In [100]:
inpData.to_csv('../tmp/inpDataBT.txt',index=True, columns=column_order, sep='\t', header=header)

### create bulInfo.txt file

In [101]:
tt=pd.to_datetime(inpData['t'][0])

In [102]:
tt=datetime.datetime.strftime(tt,'%d %b %Y %H:%M:%S')

In [103]:
dic0={'advNo':[1],'tShift':[0],'$date':tt,'land':[1],'notes':[0]}

In [104]:
bul=pd.DataFrame.from_dict(dic0)

In [105]:
bul

,$date,advNo,land,notes,tShift
0,12 Sep 2013 06:00:00,1,1,0,0


In [106]:
bul.to_csv('../tmp/bulInfo.txt',index=False, columns=['advNo','tShift','$date','land','notes'], sep='\t')

### create info.xml file

In [107]:
import xml.etree.ElementTree as et 
from xml.dom import minidom

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = et.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [108]:
info = et.Element('setexp')
et.SubElement(info, 'source').text = 'Tropical Cyclone Bulletin through GDACS/PDC'
et.SubElement(info, 'hurName').text = hurName
et.SubElement(info, 'hurId').text = hurName
et.SubElement(info, 'basin').text = ntc[0].strip()
et.SubElement(info, 'bulNo').text = '1'
et.SubElement(info, 'bulDate').text = tt
et.SubElement(info, 'n').text = '100000'
et.SubElement(info, 'fk').text = '0.81'
et.SubElement(info, 'stormsurge').text = '0'
et.SubElement(info, 'timefactor').text = '1'
et.SubElement(info, 'landfall').text = '1'

In [109]:
xmlf = minidom.parseString(prettify(info))

In [110]:
with open('../tmp/info.xml','w') as f:
      xmlf.writexml(f)

The next step is to substract translational and coriolis velocity, see [Subtract translational and Coriolis velocity.ipynb](./Subtract translational and Coriolis velocity.ipynb).